## Laboratorium 3

In [1]:
import numpy as np


In [2]:
# imports for tokenizer
from spacy.tokenizer import Tokenizer
from spacy.lang.pl import Polish
nlp = Polish()
# Create a blank Tokenizer with just the English vocab
tokenizer = Tokenizer(nlp.vocab)
# tokenizer = nlp.tokenizer


### Zadanie 1 i 2

In [107]:
class StringHelper():
    def __init__(self, str1: str, str2: str):
        self.str1 = str1
        self.str2 = str2
        self.matrix_of_changes = None
        self.levDistance = None
        self.document = None
        self.operations = []
        self.positions = []
        self.flag_to_print = True

    def is_lev(self, flag):
        if flag:
            return 1
        return 2

    def levenshtein_distance(self, str1=None, str2=None, flag = True):
        if str1 is None : str1 = self.str1
        if str2 is None : str2 = self.str2
        len1 = len(str1) + 1  # wiersze
        len2 = len(str2) + 1  # kolumny
        tab = [[0 for _ in range(len2)] for _ in range(len1)]

        for i in range(len1):
            tab[i][0] = i
        for i in range(len2):
            tab[0][i] = i

        for i in range(1, len1):
            for j in range(1, len2):
                if str1[i-1] != str2[j-1]:
                    tab[i][j] = min(tab[i-1][j-1] + self.is_lev(flag), tab[i]
                                    [j-1] + 1, tab[i-1][j] + 1)
                else:
                    tab[i][j] = min(tab[i-1][j-1], tab[i]
                                [j-1] + 1, tab[i-1][j] + 1)

        # for i in range(len(tab)):
        #     print(tab[i])
        # print("WYNIK")
        return tab, min(tab[-1])
    
    def get_path_of_operations(self):
        return self.operations

    def get_position_of_changes(self):
        return self.positions

    def add_letter(self, a, letter_to_add, position):
        if self.flag_to_print:
            str_to_print = "{}*{}*{} (adding letter {})".format(
                a[:position], letter_to_add, a[position:], letter_to_add)
            print(str_to_print)
        return "{}{}{}".format(a[:position], letter_to_add, a[position:])


    def delete_letter(self, a, letter_to_delete, position):
        if self.flag_to_print:
            str_to_print = "{}**{} (deleting letter {})".format(
                a[:position], a[position + 1:], letter_to_delete)
            print(str_to_print)
        return "{}{}".format(a[:position], a[position + 1:])


    def change_letter(self, a, old_letter, new_letter, position):
        if self.flag_to_print:
            str_to_print = "{}*{}*{} (changing letter {}->{})".format(
                a[:position], new_letter, a[position+1:], old_letter, new_letter)
            print(str_to_print)
        return "{}{}{}".format(a[:position], new_letter, a[position+1:])


    def visualizate_steps(self, str1=None, str2=None, matrix_of_changes=None, flag = True):
        # we are changing str1 -> str2
        if str1 is None : str1 = self.str1
        if str2 is None : str2 = self.str2
        if matrix_of_changes is None:
            self.matrix_of_changes, self.levDistance = self.levenshtein_distance(flag = flag)
            matrix_of_changes = self.matrix_of_changes

        y = len(str1)  # rows
        x = len(str2) # columns
            

        while x > 0 or y > 0:
            # curr_num = matrix_of_changes[y][x]
            # tmp_x = matrix_of_changes[y][x - 1] if x - 1 >= 0 else float("inf")
            # tmp_y = matrix_of_changes[y - 1][x] if y - 1 >= 0 else float("inf")
            # tmp_x_y = matrix_of_changes[y - 1][x - 1] if tmp_x != float("inf") and tmp_y != float("inf") else float("inf") 
            
            # curr_min = min(tmp_x, tmp_y, tmp_x_y)
            if matrix_of_changes[y - 1][x] + 1 == matrix_of_changes[y][x]:
                self.operations.append("DELETE")
                y -= 1
            elif matrix_of_changes[y][x - 1] + 1 == matrix_of_changes[y][x]:
                self.operations.append("ADD")
                x -= 1
            elif matrix_of_changes[y - 1][x - 1] == matrix_of_changes[y][x]:
                self.operations.append("NOTHING")
                y -= 1
                x -= 1
            else:
                self.operations.append("CHANGE")
                y -= 1
                x -= 1


            self.positions.append(x)

        self.operations.reverse()

        if self.flag_to_print:
            k = 0
            print(str1)
            for i in range(len(self.operations)):
                match self.operations[i]:
                    case "ADD":
                        str1 = self.add_letter(str1, str2[k], k)
                    case "DELETE":
                        str1 = self.delete_letter(str1, str1[k], k)
                        continue
                    case "CHANGE":
                        str1 = self.change_letter(str1, str1[k], str2[k], k)
                k += 1
                
            print(str1)
        
        self.positions.reverse()
    


### Zadanie 3 - przedstawienie wyników algorytmu dla różnych danych

In [108]:
example1 = StringHelper("los", "kloc")
example1.visualizate_steps()

los
*k*los (adding letter k)
klo*c* (changing letter s->c)
kloc


In [109]:
example2 = StringHelper("Łódź", "Lodz")
example2.visualizate_steps()

Łódź
*L*ódź (changing letter Ł->L)
L*o*dź (changing letter ó->o)
Lod*z* (changing letter ź->z)
Lodz


In [83]:
example3 = StringHelper("kwintesencja", "quintessence")
example3.visualizate_steps()

kwintesencja
*q*wintesencja (changing letter k->q)
qwinte*s*sencja (adding letter s)
qwintessen**ja (deleting letter c)
qwintessen*c*a (changing letter j->c)
qwintessenca


In [122]:
example4 = StringHelper("ATGAATCTTACCGCCTCG", "ATGAGGCTCTGGCCCCTG")
example4.visualizate_steps()

ATGAATCTTACCGCCTCG
ATGA*G*TCTTACCGCCTCG (changing letter A->G)
ATGAG*G*CTTACCGCCTCG (changing letter T->G)
ATGAGGCT*C*TACCGCCTCG (adding letter C)
ATGAGGCTCT*G*CCGCCTCG (changing letter A->G)
ATGAGGCTCTG*G*CCGCCTCG (adding letter G)
ATGAGGCTCTGGCC**CCTCG (deleting letter G)
ATGAGGCTCTGGCCCCT**G (deleting letter C)
ATGAGGCTCTGGCCCCTG


In [97]:
example5 = StringHelper("kwintesencja", "quintessence")
example5.lcs()

### Zadanie 4

In [5]:
def lcs(str1, str2):

    len1 = len(str1) + 1  # wiersze
    len2 = len(str2) + 1  # kolumny
    tab = [[0 for _ in range(len2)] for _ in range(len1)]

    for i in range(len1):
        tab[i][0] = i
    for i in range(len2):
        tab[0][i] = i

    for i in range(1, len1):
        for j in range(1, len2):
            if str1[i-1] != str2[j-1]:
                tab[i][j] = min(tab[i-1][j-1] + 2, tab[i]
                                [j-1] + 1, tab[i-1][j] + 1)
            else:
                tab[i][j] = min(tab[i-1][j-1], tab[i]
                                [j-1] + 1, tab[i-1][j] + 1)


    return (len(str1) + len(str2) - tab[-1][-1])/2

 

In [6]:
def remove_3_percentage_of_tokens(tokens):
    new_tokens = []
    for token in tokens:
        if np.random.random() > 0.03:
            new_tokens.append(token)
    return new_tokens

In [7]:
def save_tokens_to_file(tokens, filename):
    with open("{}.txt".format(filename), "w") as f:
        for token in tokens:
            f.write(token.text_with_ws)

In [115]:
filename = "romeo-i-julia-700.txt"

In [116]:
with open(filename, "r") as f:
    content = f.read()

### Zadanie 5 i 6

In [117]:
tokens = tokenizer(content)
tokens1 = remove_3_percentage_of_tokens(tokens)
tokens2 = remove_3_percentage_of_tokens(tokens)

In [118]:
save_tokens_to_file(tokens1, "text1")
save_tokens_to_file(tokens2, "text2")

### Zadanie 7

In [12]:
lcs(tokens1, tokens2)

2146.0

In [13]:
with open("text1.txt", "r") as f:
    text1 = f.read()
with open("text2.txt", "r") as f:
    text2 = f.read()

In [14]:
lcs(text1, text2)

11827.0

### Zadanie 8

In [127]:
def diff(filename1, filename2):
    with open(filename1, "r") as f:
        text1 = f.read()

    with open(filename2, "r") as f:
        text2 = f.read()

    text1 = text1.split('\n')
    text2 = text2.split('\n')

    johoho = StringHelper(text1, text2)
    johoho.flag_to_print = False
    johoho.visualizate_steps(flag = False)

    operations = johoho.get_path_of_operations()
    positions = johoho.get_position_of_changes()
    j = 0
    for i in range(len(operations)):
        match operations[i]:
            case "ADD":
                print("> ({}) {}".format(positions[i] + 1, text1[positions[i]]))
                j -= 1
            case "DELETE":
                print("< ({}) {}".format(positions[i] + j + 1, text2[positions[i] + j]))
                j += 1
    # BRUH: program się wywala gdy na końcu pliku są puste lini :L
            

### Zadanie 9

In [129]:
diff("text1.txt", "text2.txt")

> (4) tłum. Józef Paszkowski
< (4) Józef Paszkowski
> (10) 
> (11)  ESKALUS — książę panujący w Weronie
> (12)  * PARYS — młody Weroneńczyk szlachetnego krewny księcia
> (13)  MONTEKI, KAPULET — naczelnicy dwóch domów nieprzyjaznych sobie
< (10) OSOBY:
< (11)  * ESKALUS — książę panujący w Weronie
< (12)  PARYS — młody Weroneńczyk szlachetnego rodu, krewny księcia
< (13)  * MONTEKI, KAPULET — naczelnicy dwóch domów nieprzyjaznych sobie
> (17)  * BENWOLIO — synowiec Montekiego
< (17)  * — synowiec Montekiego
> (27)  * PIOTR
> (28)  * DOWÓDCA WARTY
> (29)  * PANI MONTEKI — małżonka Montekiego
< (27)  * PIOTR* DOWÓDCA WARTY
< (28)  * PANI MONTEKI małżonka Montekiego
< (29)  * PANI KAPULET — małżonka Kapuleta
< (30)  * JULIA — córka Kapuletów
> (32)  * MARTA — mamka Julii
< (33) 
> (46) Dwa rody, zacne jednako i sławne —
< (47) Do nowej zbrodni pchają złości dawne,
> (52) Pod najstraszliwszą z gwiazd, kochanków dwoje;
> (53) Po pełnym przygód nieszczęśliwych bycie
> (54) Śmierć ich stłumił